In [1]:
import numpy as np
from numpy.core.fromnumeric import _all_dispatcher
import pandas as pd
import joblib
np.random.seed(2021)
import warnings
warnings.filterwarnings('ignore')

# df --> whole training set

In [2]:
df = pd.read_csv("train_modified.gz", compression='gzip', header='infer')
Y = df['click']
# discard some columns
# unused_cols = ["id", 'site_id', 'app_id']
# df.drop(unused_cols, axis=1, inplace=True)

In [3]:
df

,click,hour,C1,banner_pos,site_id,site_domain,site_category,device_id,device_ip,device_model,...,C19,C20,C21,day_of_week,device_ip_count,device_id_count,hour_count,user,hourly_user_count,click_history
0,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,ddd2926e,44956a24,...,3,-1,67,1,7647,2533255,140117,ddd2926e44956a24,4.0,first string
1,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,96809ac8,711ee120,...,3,85,67,1,7,2533255,140117,96809ac8711ee120,3.0,first string
2,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,b3cf8def,8a4875bd,...,3,85,67,1,2,2533255,140117,b3cf8def8a4875bd,2.0,first string
3,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,e8275b8f,6332421a,...,3,85,67,1,6,2533255,140117,e8275b8f6332421a,2.0,first string
4,0,0,5,1,a2af7bee,cbee4b41,72722551,a99f214a,9644d0bf,779d90c2,...,3,-1,145,1,31,2533255,140117,9644d0bf779d90c2,15.0,first string
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683782,0,1,5,1,c12ebe86,c1aa3c04,74073276,a99f214a,8546df25,67fb3069,...,7,-1,145,2,229,2533255,147294,8546df2567fb3069,5.0,1110
2683783,0,1,5,1,c12ebe86,c1aa3c04,74073276,a99f214a,98e4ada3,b314d7b9,...,131,-1,49,2,18,2533255,147294,98e4ada3b314d7b9,4.0,010
2683784,1,1,5,0,5bb07e04,b256a9bc,f66779e6,a99f214a,Unknown,76dc4769,...,135,-1,11,2,378614,2533255,147294,dc38aa0776dc4769,1.0,first string
2683785,0,1,5,1,c12ebe86,c1aa3c04,74073276,a99f214a,e5693fd8,7120e05e,...,7,-1,11,2,9,2533255,147294,e5693fd87120e05e,4.0,0000010


### df_train --> our own training set

In [4]:
df_train = pd.read_csv("train_df.gz", compression='gzip', header='infer')
Y_train = df_train['click']
df_train

,click,hour,C1,banner_pos,site_id,site_domain,site_category,device_id,device_ip,device_model,...,C19,C20,C21,day_of_week,device_ip_count,device_id_count,hour_count,user,hourly_user_count,click_history
0,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,ddd2926e,44956a24,...,3,-1,67,1,5539,1936483,93115,ddd2926e44956a24,3,first string
1,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,96809ac8,711ee120,...,3,85,67,1,7,1936483,93115,96809ac8711ee120,3,first string
2,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,b3cf8def,8a4875bd,...,3,85,67,1,2,1936483,93115,b3cf8def8a4875bd,2,first string
3,0,0,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,e8275b8f,6332421a,...,3,85,67,1,5,1936483,93115,e8275b8f6332421a,2,first string
4,0,0,5,1,a2af7bee,cbee4b41,72722551,a99f214a,9644d0bf,779d90c2,...,3,-1,145,1,31,1936483,93115,9644d0bf779d90c2,15,first string
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051079,1,15,5,0,f63cbdb9,16365701,74073276,a99f214a,57cd4006,711ee120,...,3,-1,209,1,5334,1936483,113179,57cd4006711ee120,21,0000101010100101000101000100101010110010000001...
2051080,1,15,5,1,8480f5b7,3eda54e5,74073276,a99f214a,Unknown,84ebbcd4,...,7,-1,11,1,296165,1936483,113179,bc33a2d584ebbcd4,1,first string
2051081,1,15,5,0,078d3465,dd641cc7,8fd0aea4,a99f214a,139b68fd,76dc4769,...,3,85,209,1,2,1936483,113179,139b68fd76dc4769,2,1
2051082,0,15,5,1,a831e579,437312ac,74073276,a99f214a,57bf6be0,d787e91b,...,3,-1,31,1,4,1936483,113179,57bf6be0d787e91b,1,first string


In [5]:
Y_train[:1000]

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: click, Length: 1000, dtype: int64

### Optimal catboost

In [15]:
from sklearn.model_selection import train_test_split

df_cat = df_train.copy()
unused_cols = ['site_id', 'click']
df_cat.drop(unused_cols, axis=1, inplace=True)
cut_off = int(len(df_cat) * 0.7)
X_train_cat = df_cat.iloc[:cut_off, :]
X_test_cat = df_cat.iloc[cut_off:, :]
y_train_cat = Y_train[:cut_off]
y_test_cat = Y_train[cut_off:]

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer

categorical_f = ['C1', 'banner_pos','site_domain','site_category','device_id','device_ip','device_model',
                 'device_type','device_conn_type', 'C14','C15','C16','C17','C18', 'C19','C20','C21',
                 'day_of_week', 'user', 'click_history']

cat1 = CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7,loss_function='Logloss', 
                          cat_features=categorical_f,verbose=False)

param = {
    'iterations': Integer(10, 1000),
    'depth': Integer(1, 8),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'random_strength': Real(1e-9, 10, 'log-uniform'),
    'bagging_temperature': Real(0.0, 1.0),
    'border_count': Integer(1, 255),
    'l2_leaf_reg': Integer(2, 30),
    'scale_pos_weight':Real(0.01, 1.0, 'uniform')
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# log-uniform: understand as search over p = exp(x) by varying x
opt_c = BayesSearchCV(
    cat1,
    param,
    scoring = LogLoss,
    n_iter=64,
    cv=5,
    random_state=42
)

# executes bayesian optimization
opt_c.fit(X_train_cat, y_train_cat)

In [ ]:
opt_c.best_score_

In [ ]:
opt_c.best_params_

### optimal xgboost

In [6]:
df_xgb = df_train.copy()

def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self

df_xgb = convert_obj_to_int(df_xgb)
cut_off = int(len(df_xgb) * 0.7)
X_train_xgb = df_xgb.iloc[:cut_off, :]
X_test_xgb = df_xgb.iloc[cut_off:, :]
y_train_xgb = Y_train[:cut_off]
y_test_xgb = Y_train[cut_off:]

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer

xgb1 = XGBClassifier(max_depth=3,learning_rate=0.1, alpha=0, colsample_bytree = 0.5,
                     subsample=0.1,n_estimators=100,gamma=0)

param = {
    'iterations': Integer(10, 400),
    'max_depth': Integer(3, 8, 'uniform'),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'alpha': Real(0, 1.0, 'uniform'),
    'colsample_bytree' : Real(0.5,1.0, 'uniform'),
    'subsample': Real(0.1, 1.0, 'uniform'),
    'n_estimnators': Integer(100, 1000, 'uniform'),
    'gamma': Real(0, 1.0, 'uniform')
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# log-uniform: understand as search over p = exp(x) by varying x
opt_x = BayesSearchCV(
    xgb1,
    param,
    scoring = LogLoss,
    n_iter=40,
    cv=5,
    random_state=42
)

# executes bayesian optimization
opt_x.fit(X_train_xgb, y_train_xgb)

[22:22:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:23:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:23:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an i

[22:28:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:28:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:29:05] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[22:33:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:33:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:33:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:34:00] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[22:38:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:38:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:38:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:39:03] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[22:43:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:43:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:43:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:43:57] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

...

[23:24:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:24:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:24:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:25:20] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[23:30:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:30:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:30:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:30:59] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[23:34:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:34:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:34:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:34:32] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

[23:38:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:38:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { iterations, n_estimnators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:38:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:39:14] WARNING: /Users/travis/build/dmlc/xgboost/src/lear

BayesSearchCV(cv=5,
              estimator=XGBClassifier(alpha=0, base_score=None, booster=None,
                                      colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=0.5, gamma=0,
                                      gpu_id=None, importance_type='gain',
                                      interaction_constraints=None,
                                      learning_rate=0.1, max_delta_step=None,
                                      max_depth=3, min_child_weight=None,
                                      missing=nan, monotone_constraints=None,
                                      n_estimators=100, n_job...
                             'iterations': Integer(low=10, high=400, prior='uniform', transform='identity'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='identity'),
                             'max_depth': Intege

In [13]:
opt_x.best_score_

-1.5349627176835216e-06

In [14]:
opt_x.best_params_

OrderedDict([('alpha', 0.0),
             ('colsample_bytree', 1.0),
             ('gamma', 0.0),
             ('iterations', 400),
             ('learning_rate', 0.27950642975302614),
             ('max_depth', 5),
             ('n_estimnators', 100),
             ('subsample', 1.0)])

### optimal lightgbm

In [ ]:
df_lgb = df_train.copy()

def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self

df_lgb = convert_obj_to_int(df_lgb)
cut_off = int(len(df_lgb) * 0.7)
X_train_lgb = df_lgb.iloc[:cut_off, :]
X_test_lgb = df_lgb.iloc[cut_off:, :]
y_train_lgb = Y_train[:cut_off]
y_test_lgb = Y_train[cut_off:]

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer, auc, log_loss, roc_auc_score

import lightgbm as lgb

# categorical_f = [ca for ca in X_train.columns if X_train[ca].dtype == 'object']
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', metric='binary_logloss', bagging_freq=5)

param = {
    'max_depth': Integer(3, 7),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'feature_fraction': Real(0.2, 0.9, 'uniform'),
    'bagging_fraction': Real(0.2, 0.9, 'log-uniform'),
    'max_bin': Integer(20, 255, 'uniform'),
    'n_estimators': Integer(100, 1000, 'uniform'),
    'num_leaves': Integer(24, 80, 'uniform'),
    'min_sum_hessian_in_leaf':Integer(0,100, 'uniform'),
}

# log-uniform: understand as search over p = exp(x) by varying x
opt_l = BayesSearchCV(
    lgb_model,
    param,
    scoring = LogLoss,
    n_iter=32,
    cv=5,
    random_state=42,
    verbose=5
)

# executes bayesian optimization
opt_l.fit(X_train_lgb, y_train_lgb)

## K-fold

In [65]:
df_opt = df_train.copy()
df_opt_2 = df_train.copy()
unused_cols = ['site_id', 'click']
df_opt.drop(unused_cols, axis=1, inplace=True)
df_opt_2.drop(unused_cols, axis=1, inplace=True)

In [66]:
best_p_c = {
    'iterations': 400,
    'depth': 4,
    'learning_rate': 1.0,
    'random_strength': 10,
    'bagging_temperature': 1.0,
    'border_count': 255,
    'l2_leaf_reg': 30,
    'scale_pos_weight':0.86438264586532
}

best_p_x = {
    'alpha':0.0,
    'colsample_bytree':1.0,
    'gamma':0.0,
    'iterations':400,
    'learning_rate':0.27950642975302614,
    'max_depth':5,
    'n_estimnators':100,
    'subsample':1.0
}

best_p_l = {
    'boosting_type':'gbdt', 
    'objective': 'binary', 
    'metric':'binary_logloss', 
    'bagging_freq': 5, 
    'max_depth': 7,
    'learning_rate': 0.11304216699488043,
    'feature_fraction': 0.5066204305086464,
    'bagging_fraction': 0.6657456066570288,
    'max_bin': 188,
    'n_estimators': 482,
    'num_leaves': 60,
    'min_sum_hessian_in_leaf':72
}

In [ ]:
from sklearn.model_selection import KFold

# preds = np.zeros(test.shape[0])
logloss_c=[]
logloss_x=[]  # list contains rmse for each fold
logloss_l=[]
proba_cat_li=[]
proba_xgb_li=[]
proba_lgb_li=[]
y_li = []
n=0
df_opt = convert_obj_to_int(df_opt)

categorical_f = ['C1', 'banner_pos','site_domain','site_category','device_id','device_ip','device_model',
                 'device_type','device_conn_type', 'C14','C15','C16','C17','C18', 'C19','C20','C21',
                 'day_of_week', 'user', 'click_history']

kf = KFold(n_splits=10,random_state=42,shuffle=True)
#     preds+=xgb.predict(test[columns])/kf.n_splits
for train_idx, test_idx in kf.split(df_opt,Y_train):
    X_tr,X_val=df_opt_2.iloc[train_idx],df_opt_2.iloc[test_idx]
    y_tr,y_val=Y_train.iloc[train_idx],Y_train.iloc[test_idx]
    
    X_tr_hash,X_val_hash=df_opt.iloc[train_idx],df_opt.iloc[test_idx]
    
    cat = CatBoostClassifier(**best_p_c,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
    cat.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    proba_cat = cat.predict_proba(X_val.values)[:, 1]
    proba_cat_li.append(proba_cat)
    logloss_c.append(log_loss(y_val, proba_cat))
    print(n+1,logloss_c[n])
    
    xgb = XGBClassifier(**best_p_x, loss_function='Logloss')
    xgb.fit(X_tr_hash,y_tr,eval_set=[(X_val_hash,y_val)],early_stopping_rounds=100,verbose=False)
    proba_xgb = xgb.predict_proba(X_val_hash.values)[:, 1]
    proba_xgb_li.append(proba_xgb)
    logloss_x.append(log_loss(y_val, proba_xgb))
    print(n+1,logloss_x[n])
    
    lgb = lgb.LGBMClassifier(best_p_l, loss_function='Logloss')
    lgb.fit(X_tr_hash,y_tr,eval_set=[(X_val_hash,y_val)],early_stopping_rounds=100, verbose=False)
    proba_lgb = lgb.predict_proba(X_val_hash.values)[:, 1]
    proba_lgb_li.append(proba_lgb)
    logloss_l.append(log_loss(y_val, proba_lgb))
    print(n+1,logloss_l[n])
    n+1
    
    y_li.append(y_val)

In [20]:
logloss_c

[0.42108933834823564,
 0.4245085451378775,
 0.4227249105200798,
 0.4242787822622415,
 0.42484490639534866,
 0.4231111782452581,
 0.42411352536754104,
 0.4226289177814561,
 0.42166378761430084,
 0.42496272631344667]

In [40]:
weights_range = [0.001, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
       0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.999]

In [ ]:
%%timeit
# bag of models in Catboost, xgboost, lightgbm
model_num = 3
cv_num = 10
#create placeholder for results table
output_wts_li = []
for i in range(cv_num):
    output_wts = np.zeros((len(X_val),model_num+1))
    output_wts_li.append(output_wts)

import itertools
for i in range(cv_num):
    j=0
    for a,b,c in itertools.product(weights_range, repeat=model_num):
        #get combination of weights, sum to 100%
        sum_w = np.array([a,b,c]).sum()
        wts = np.array([a,b,c]) / sum_w
        
        final_proba = np.zeros((len(X_val), ))
        #get oof combination for weights
        final_proba+=proba_cat_li[i] * wts[0]
        final_proba+=proba_xgb_li[i] * wts[1]
        final_proba+=proba_lgb_li[i] * wts[2]

        #get error and put into output table
        output_wts_li[i][j,model_num] = log_loss(y_li[i], final_proba[i])

        #record the associated weights
        output_wts[i][j,0:model_num] = wts

        j+=1

In [36]:
f = np.ones((10, 205108))

In [37]:
f[0]+=y_val * 100

In [38]:
f[0]+=y_val * 100

In [39]:
f[0]

array([201., 201.,   1., ..., 201.,   1., 201.])

In [46]:
import itertools
for a,b,c in itertools.product(weights_range, repeat=3):
    #get combination of weights, sum to 100%
    sum_w = np.array([a,b,c]).sum()
    wts = np.array([a,b,c]) / sum_w
    print(wts[2])

0.3333333333333333
0.9615384615384615
0.9803921568627451
0.9868421052631579
0.9900990099009901
0.9920634920634921
0.9933774834437086
0.9943181818181818
0.9950248756218906
0.995575221238938
0.9960159362549801
0.9963768115942029
0.9966777408637874
0.9969325153374233
0.9971509971509972
0.9973404255319149
0.9975062344139651
0.9976525821596244
0.9977827050997783
0.9978991596638656
0.9980019980019981
0.01923076923076923
0.49504950495049505
0.662251655629139
0.7462686567164178
0.7968127490039841
0.8305647840531561
0.8547008547008548
0.8728179551122195
0.8869179600886918
0.8982035928143712
0.9074410163339383
0.9151414309484193
0.9216589861751151
0.927246790299572
0.9320905459387483
0.9363295880149812
0.9400705052878966
0.9433962264150942
0.9463722397476341
0.9490509490509491
0.9514285714285714
0.00980392156862745
0.3311258278145695
0.4975124378109453
0.5976095617529881
0.6644518272425248
0.7122507122507123
0.7481296758104737
0.7760532150776054
0.7984031936127745
0.8166969147005444
0.8319467554

0.6086956521739131
0.625
0.64
0.653846153846154
0.6666666666666666
0.6785714285714286
0.68944099378882
0.001996007984031936
0.09090909090909091
0.16666666666666669
0.23076923076923075
0.28571428571428575
0.3333333333333333
0.37499999999999994
0.4117647058823529
0.4444444444444445
0.4736842105263158
0.5
0.5238095238095238
0.5454545454545454
0.5652173913043479
0.5833333333333334
0.6
0.6153846153846154
0.6296296296296295
0.6428571428571429
0.6551724137931034
0.6664442961974649
0.0018148820326678765
0.08333333333333333
0.15384615384615385
0.21428571428571425
0.26666666666666666
0.3125
0.3529411764705882
0.38888888888888884
0.42105263157894735
0.45
0.47619047619047616
0.5
0.5217391304347826
0.5416666666666666
0.5599999999999999
0.5769230769230769
0.5925925925925926
0.6071428571428572
0.6206896551724137
0.6333333333333333
0.644932214331827
0.0016638935108153079
0.07692307692307693
0.14285714285714288
0.19999999999999998
0.25
0.29411764705882354
0.33333333333333337
0.3684210526315789
0.4
0.42

0.19999999999999998
0.22580645161290325
0.25
0.27272727272727276
0.29411764705882354
0.31428571428571433
0.33333333333333337
0.35135135135135137
0.3684210526315789
0.38461538461538464
0.4
0.41463414634146345
0.42857142857142855
0.4418604651162791
0.45429740791268763
0.0007993605115907275
0.038461538461538464
0.07407407407407407
0.10714285714285715
0.13793103448275862
0.16666666666666666
0.1935483870967742
0.21874999999999997
0.24242424242424246
0.2647058823529412
0.2857142857142857
0.3055555555555556
0.3243243243243243
0.34210526315789475
0.358974358974359
0.375
0.39024390243902446
0.4047619047619047
0.4186046511627907
0.43181818181818177
0.44419742107603377
0.0007692307692307693
0.037064492216456635
0.07147962830593281
0.10351966873706005
0.133422281521014
0.16139444803098774
0.18761726078799248
0.21224984839296543
0.23543260741612718
0.2572898799313894
0.27793218454697055
0.29745808545159547
0.315955766192733
0.3335043612108774
0.35017508754377186
0.36603221083455345
0.38113387327298

0.45
0.4634146341463415
0.4759409242496426
0.0008688097306689837
0.04166666666666667
0.08
0.11538461538461539
0.14814814814814817
0.17857142857142858
0.20689655172413793
0.2333333333333333
0.2580645161290323
0.28125000000000006
0.30303030303030304
0.3235294117647059
0.34285714285714286
0.36111111111111116
0.3783783783783784
0.39473684210526316
0.4102564102564103
0.425
0.4390243902439025
0.45238095238095244
0.46486738017682644
0.0008326394671107412
0.04
0.07692307692307693
0.11111111111111112
0.14285714285714288
0.1724137931034483
0.19999999999999998
0.22580645161290325
0.25
0.27272727272727276
0.29411764705882354
0.31428571428571433
0.33333333333333337
0.35135135135135137
0.3684210526315789
0.38461538461538464
0.4
0.41463414634146345
0.42857142857142855
0.4418604651162791
0.45429740791268763
0.0007993605115907275
0.038461538461538464
0.07407407407407407
0.10714285714285715
0.13793103448275862
0.16666666666666666
0.1935483870967742
0.21874999999999997
0.24242424242424246
0.2647058823529

0.07692307692307693
0.11111111111111112
0.14285714285714288
0.1724137931034483
0.19999999999999998
0.22580645161290325
0.25
0.27272727272727276
0.29411764705882354
0.31428571428571433
0.33333333333333337
0.35135135135135137
0.3684210526315789
0.38461538461538464
0.4
0.41463414634146345
0.42857142857142855
0.4418604651162791
0.45429740791268763
0.0007993605115907275
0.038461538461538464
0.07407407407407407
0.10714285714285715
0.13793103448275862
0.16666666666666666
0.1935483870967742
0.21874999999999997
0.24242424242424246
0.2647058823529412
0.2857142857142857
0.3055555555555556
0.3243243243243243
0.34210526315789475
0.358974358974359
0.375
0.39024390243902446
0.4047619047619047
0.4186046511627907
0.43181818181818177
0.44419742107603377
0.0007686395080707151
0.03703703703703704
0.07142857142857144
0.10344827586206898
0.13333333333333336
0.16129032258064518
0.1875
0.21212121212121213
0.23529411764705888
0.2571428571428572
0.2777777777777778
0.29729729729729737
0.3157894736842105
0.333333

0.07894736842105263
0.10256410256410257
0.125
0.14634146341463417
0.16666666666666666
0.186046511627907
0.20454545454545453
0.2222222222222222
0.23913043478260873
0.2553191489361702
0.27083333333333337
0.2857142857142857
0.3
0.3137254901960785
0.3269230769230769
0.339622641509434
0.3518518518518518
0.3634048744998181
0.000555247084952804
0.02702702702702703
0.05263157894736842
0.07692307692307693
0.1
0.12195121951219513
0.14285714285714285
0.1627906976744186
0.18181818181818182
0.2
0.2173913043478261
0.23404255319148937
0.25
0.2653061224489796
0.27999999999999997
0.29411764705882354
0.3076923076923077
0.32075471698113206
0.3333333333333333
0.34545454545454546
0.35691318327974275
0.0005402485143165856
0.02631578947368421
0.05128205128205128
0.075
0.09756097560975609
0.11904761904761904
0.13953488372093023
0.15909090909090906
0.17777777777777778
0.19565217391304346
0.2127659574468085
0.22916666666666666
0.24489795918367344
0.26
0.27450980392156865
0.28846153846153844
0.3018867924528302
0